<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_006_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Ajuste de hiperparámetros con búsqueda aleatoria (Randomized Search)</font>**

<p align="justify">
♥ En el Colab anterior, mostramos cómo usar un enfoque de grid-search para buscar los mejores hiperparámetros que maximicen el rendimiento de generalización de un modelo predictivo.
<br><br>
Sin embargo, un enfoque de grid-search tiene limitaciones. No se escala cuando aumenta el número de parámetros a ajustar.
<br><br>
Además, la grid-search impondrá una regularidad durante la búsqueda que podría ser problemática.
En este Colab, presentaremos otro método para ajustar hiperparámetros denominado randomized search (búsqueda aleatoria) .

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px

 ## **<font color="DeepPink">Carga del conjunto de datos</font>**

In [ ]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")
adult_census.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


 ## **<font color="DeepPink">Separamos la variable objetivo y las variables explicativas</font>**

<p align="justify">
👀 Asignamos a un objeto la variable objetivo:
</p>


In [ ]:
target_name = "class"
y = adult_census[target_name]
y

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

<p align="justify">
👀 Eliminamos de nuestros datos la variable objetivo y la columna <code>education-num</code> que duplica la información de la columna <code>education</code>.

In [ ]:
X = adult_census.drop(columns=[target_name, "education-num"])
X.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


 ## **<font color="DeepPink">Conjunto de entrenamiento y conjunto de prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 ## **<font color="DeepPink">Pipeline</font>**

<p align="justify">
✅ Definiremos un Pipeline que va a manejar características categóricas.

In [ ]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

<p align="justify">
✅ Seleccionamos las columnas categoricas.

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

<p align="justify">
✅ Ahora codificamos con <code>OrdinalEncoder</code>.

In [ ]:
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

<p align="justify">
✅ Luego usamos un <code>ColumnTransformer</code> para seleccionar las columnas categóricas y les aplicamos el <code>OrdinalEncoder</code>

In [ ]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

In [ ]:
preprocessor

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat_preprocessor',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

<p align="justify">
✅ Finalmente, usamos un clasificador basado en árboles (histogram gradient-boosting) para predecir si una persona gana o no más de 50 k$ al año.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [ ]:
model = Pipeline([("preprocessor", preprocessor),
                  ("classifier", HistGradientBoostingClassifier(random_state=42,
                                                                max_leaf_nodes=4))])

In [ ]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

 # **<font color="DeepPink">Ajuste mediante randomized search</font>**

<p align="justify">
Con el estimador <code>GridSearchCV</code>, los parámetros deben especificarse explícitamente. Ya mencionamos que explorar una gran cantidad de valores para diferentes parámetros será rápidamente intratable.
<br><br>
En su lugar, podemos generar aleatoriamente los parámetros candidatos. De hecho, tal enfoque evita la regularidad de la red. Por lo tanto, agregar más evaluaciones puede aumentar la resolución en cada dirección. Este es el caso en la situación frecuente en la que la elección de algunos hiperparámetros no es muy importante, como ocurre con el hiperparámetro 2 en la figura siguiente.

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Optimizacion-001.png?raw=true" width="450">
</p>


<p align="justify">
De hecho, el número de puntos de evaluación debe dividirse entre los dos hiperparámetros diferentes. Con una grilla, el peligro es que la región de buenos hiperparámetros quede entre la línea de la grilla: esta región está alineada con la grilla dado que el hiperparámetro $2$ tiene una influencia débil.
<br><br>
Más bien, la búsqueda estocástica muestreará el hiperparámetro $1$ independientemente del hiperparámetro $2$ y encontrará la región óptima.
<br><br>
La clase <code>RandomizedSearchCV</code> permite dicha búsqueda estocástica. Se usa de manera similar a <code>GridSearchCV</code>, pero es necesario especificar las distribuciones de muestreo en lugar de los valores de los parámetros.
<br><br>
Por ejemplo, dibujaremos candidatos usando una distribución logarítmica uniforme porque los parámetros que nos interesan toman valores positivos con una escala logarítmica natural ($.1$ es tan cercano a $1$ como $10$).
<br><br>
La búsqueda aleatoria (con <code>RandomizedSearchCV</code>) suele ser beneficiosa en comparación con la búsqueda en cuadrícula (con <code>GridSearchCV</code>) para optimizar $3$ o más hiperparámetros.

<p align="justify">
Optimizaremos otros $3$ parámetros además de los que optimizamos en el Colab que muestra <code>GridSearchCV</code>:

- `l2_regularización`: corresponde a la regularización;
- `min_samples_leaf`: corresponde al número mínimo de muestras requeridas en una hoja;
- `max_bins`: corresponde al número máximo de bins para construir los histogramas.

<p align="justify">
Recordamos el significado de los $2$ parámetros restantes:

- `learning_rate`: corresponde a la velocidad que el boosting corregirá los residuos en cada iteración;
- `max_leaf_nodes`: corresponde al número máximo de hojas para cada árbol del conjunto.

<p align="justify">
<code>scipy.stats.loguniform</code> se puede usar para generar números float. Para generar valores aleatorios para parámetros con valores enteros (por ejemplo, <code>min_samples_leaf</code>), podemos adaptar lo siguiente:

In [ ]:
from scipy.stats import loguniform

In [ ]:
class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

<p align="justify">
Ahora, podemos definir la búsqueda aleatoria usando las diferentes distribuciones. Ejecutar $10$ iteraciones de validación cruzada de $5$ veces para parametrizaciones aleatorias de este modelo en este conjunto de datos puede llevar de $10$ segundos a varios minutos, según la velocidad de la computadora y la cantidad de procesadores disponibles.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_distributions = {'classifier__l2_regularization': loguniform(1e-6, 1e3),
                       'classifier__learning_rate': loguniform(0.001, 10),
                       'classifier__max_leaf_nodes': loguniform_int(2, 256),
                       'classifier__min_samples_leaf': loguniform_int(1, 100),
                       'classifier__max_bins': loguniform_int(2, 255),}

In [ ]:
model_random_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=20,
    cv=5, verbose=1,)

In [ ]:
model_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                sparse_threshold=0,
                                                                transformers=[('cat_preprocessor',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                              unknown_value=-1),
                                                                               ['workclass',
                                                                                'education',
                                                                                'marital-status',
                                                                                'occupation',
                                                                                'relationship',
                                                                                'race',
                                                                                'sex',
                                                                                'native-country'])])),
                                             ('classifier',
                                              Hi...
                   param_distributions={'classifier__l2_regularization': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fe441d31300>,
                                        'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fe441d30c40>,
                                        'classifier__max_bins': <__main__.loguniform_int object at 0x7fe441d33910>,
                                        'classifier__max_leaf_nodes': <__main__.loguniform_int object at 0x7fe441d32d70>,
                                        'classifier__min_samples_leaf': <__main__.loguniform_int object at 0x7fe441d33610>},
                   verbose=1)

<p align="justify">
👀 Podemos computar el <code>Accuracy</code> score sobre el conjunto de prueba.

In [ ]:
accuracy = model_random_search.score(X_test, y_test)

In [ ]:
print("")
print(f"The test accuracy score of the best model is "
      f"{accuracy:.2f}")


The test accuracy score of the best model is 0.87


In [ ]:
from pprint import pprint

In [ ]:
print("")
print("The best parameters are:")
pprint(model_random_search.best_params_)


The best parameters are:
{'classifier__l2_regularization': 719.14606696981,
 'classifier__learning_rate': 0.06581555639826125,
 'classifier__max_bins': 237,
 'classifier__max_leaf_nodes': 30,
 'classifier__min_samples_leaf': 2}


<p align="justify">
👀 Podemos inspeccionar el resultado, usando el atributo <code>cv_results</code>.

In [ ]:
column_results = [
    f"param_{name}" for name in param_distributions.keys()]

In [ ]:
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]

In [ ]:
cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False)

In [ ]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

In [ ]:
cv_results = cv_results.rename(shorten_param, axis=1)
cv_results.head()

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
9,719.146067,0.065816,30,2,237,0.857662,0.002716,1
13,0.000007,0.039471,39,9,49,0.856051,0.002766,2
0,3.030224,0.082923,5,58,59,0.855969,0.003865,3
2,0.000007,0.111775,71,1,50,0.853376,0.004185,4
4,0.003731,0.024465,36,6,27,0.852830,0.003146,5


<p align="justify">
Hay que tener en cuenta que el ajuste está limitado por la cantidad de combinaciones diferentes de parámetros que se puntúan mediante la búsqueda aleatoria. De hecho, puede haber otros conjuntos de parámetros que conduzcan a resultados de generalización similares o mejores, pero que no se probaron en la búsqueda.
<br><br>
En la práctica, una búsqueda aleatoria de hiperparámetros suele ejecutarse con un gran número de iteraciones.

 # **<font color="DeepPink">Visualizacion</font>**

<p align="justify">
Como tenemos más de $2$ parámetros en nuestra búsqueda aleatoria, no podemos visualizar los resultados usando un mapa de calor. Se podría hacer por parejas, pero se tendría una proyección bidimensional de un problema multidimensional y eso puede conducir a una interpretación incorrecta de las puntuaciones.

In [ ]:
df = pd.DataFrame(
    {"max_leaf_nodes": cv_results["max_leaf_nodes"],
     "learning_rate": cv_results["learning_rate"],
     "score_bin": pd.cut(cv_results["mean_test_score"], bins=np.linspace(0.5, 1.0, 6)),
    })

In [ ]:
df.head()

,max_leaf_nodes,learning_rate,score_bin
9,30,0.065816,"(0.8, 0.9]"
13,39,0.039471,"(0.8, 0.9]"
0,5,0.082923,"(0.8, 0.9]"
2,71,0.111775,"(0.8, 0.9]"
4,36,0.024465,"(0.8, 0.9]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 9 to 8
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   max_leaf_nodes  20 non-null     object  
 1   learning_rate   20 non-null     object  
 2   score_bin       20 non-null     category
dtypes: category(1), object(2)
memory usage: 752.0+ bytes


In [ ]:
df.score_bin.value_counts()

(0.8, 0.9]    12
(0.7, 0.8]     8
(0.5, 0.6]     0
(0.6, 0.7]     0
(0.9, 1.0]     0
Name: score_bin, dtype: int64

In [ ]:
df.score_bin = df.score_bin.astype(object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 9 to 8
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   max_leaf_nodes  20 non-null     object
 1   learning_rate   20 non-null     object
 2   score_bin       20 non-null     object
dtypes: object(3)
memory usage: 640.0+ bytes


In [ ]:
df.score_bin.value_counts()

(0.8, 0.9]    12
(0.7, 0.8]     8
Name: score_bin, dtype: int64

In [ ]:
px.scatter(df,
           x="max_leaf_nodes",
           y="learning_rate",
           color="score_bin",
           template="gridon")

<p align="justify">
👀 En el gráfico anterior, vemos que los valores de mayor rendimiento se ubican en un rango de tasa de aprendizaje entre 0,01 y 1,0, pero no tenemos control sobre cómo interactúan los otros hiperparámetros con dichos valores para la tasa de aprendizaje.
<br><br>
En su lugar, podemos visualizar todos los hiperparámetros al mismo tiempo utilizando un gráfico de coordenadas paralelas.

In [ ]:
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 9 to 8
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   l2_regularization  20 non-null     object 
 1   learning_rate      20 non-null     object 
 2   max_leaf_nodes     20 non-null     object 
 3   min_samples_leaf   20 non-null     object 
 4   max_bins           20 non-null     object 
 5   mean_test_score    20 non-null     float64
 6   std_test_score     20 non-null     float64
 7   rank_test_score    20 non-null     int32  
dtypes: float64(2), int32(1), object(5)
memory usage: 1.3+ KB


In [ ]:
cv_results = cv_results.astype("float")

In [ ]:
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 9 to 8
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   l2_regularization  20 non-null     float64
 1   learning_rate      20 non-null     float64
 2   max_leaf_nodes     20 non-null     float64
 3   min_samples_leaf   20 non-null     float64
 4   max_bins           20 non-null     float64
 5   mean_test_score    20 non-null     float64
 6   std_test_score     20 non-null     float64
 7   rank_test_score    20 non-null     float64
dtypes: float64(8)
memory usage: 1.4 KB


In [ ]:
px.parallel_coordinates(cv_results,
                        color="mean_test_score",
                        color_continuous_scale=px.colors.sequential.Viridis)

<p align="justify">
🛑 <b>Nota</b>: Transformamos la mayoría de los valores de los ejes tomando $log10$ o $log2$ para distribuir los rangos activos y mejorar la legibilidad de la gráfica.

In [ ]:
fig = px.parallel_coordinates(
    cv_results.rename(shorten_param, axis=1).apply(
        {"learning_rate": np.log10,
         "max_leaf_nodes": np.log2,
         "max_bins": np.log2,
         "min_samples_leaf": np.log10,
         "l2_regularization": np.log10,
         "mean_test_score": lambda x: x,}),
    color="mean_test_score",
    color_continuous_scale=px.colors.sequential.Viridis,)

fig.show()

<p align="justify">
El gráfico de coordenadas paralelas mostrará los valores de los hiperparámetros en diferentes columnas, mientras que la métrica de rendimiento está codificada por colores. Por lo tanto, podemos inspeccionar rápidamente si hay un rango de hiperparámetros que funcionan o no.
<br><br>
Es posible seleccionar un rango de resultados haciendo clic y manteniendo presionado cualquier eje del gráfico de coordenadas paralelas. Luego puede deslizar (mover) la selección de rango y cruzar dos selecciones para ver las intersecciones. Podemos deshacer una selección haciendo clic una vez más en el mismo eje.
<br><br>
En particular para esta búsqueda de hiperparámetros, es interesante confirmar que las líneas amarillas (modelos de mejor rendimiento) alcanzan valores intermedios para la tasa de aprendizaje, es decir, valores de marca entre $-2$ y $0$ que corresponden a valores de tasa de aprendizaje de $0,01$ a $1,0$ una vez que invertimos la transformación $log10$ para ese eje.
<br><br>
Pero ahora también podemos observar que no es posible seleccionar los modelos de mayor rendimiento seleccionando líneas en el eje <code>max_bins</code> con valores entre $1$ y $3$.
<br><br>
Los otros hiperparámetros no son muy sensibles. Podemos comprobar que si seleccionamos los valores del eje <code>learning_rate</code> entre $-1,5$ y $-0,5$ y los valores de <code>max_bins</code> entre $5$ y $8$, siempre seleccionamos los modelos de mejor rendimiento, independientemente de los valores de los otros hiperparámetros.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Generamos un Pipeline.
<br>
✅ Separamos en un conjunto de entrenamiento y un conjunto de prueba.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo a través del randomized search.
<br>
✅ Visualizamos los valores de los hiperparámetros con un gráfico de coordenadas paralelas.
<br><br>
En particular, observamos que algunos hiperparámetros tienen muy poco impacto en el puntaje de validación cruzada, mientras que otros deben ajustarse dentro de un rango específico para obtener modelos con buena precisión predictiva.

<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
